In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import json

In [ ]:
def read_in_data(dir_str, ref_json):

    id_to_img = {}
    df = pd.DataFrame(columns=['id', 'image_id', 'bbox', 'category_id', 'img'])

    for line in ref_json["images"]:
        filename = line["file_name"]
        img = Image.open(dir_str + "/" + filename)
        id_to_img[line["id"]] = img

    for box in ref_json["annotations"]:
        box_coords = box["bbox"]
        # im1 = im.crop((left, top, right, bottom))
        # [lewy górny róg, szerokość, wysokość]
        box_coords = [box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[1] + box_coords[3]]
        
        image = id_to_img[box["image_id"]].crop(box_coords)
        if len(image.getbands()) > 3:
            image_ = Image.new("RGB", image.size, (255, 255, 255))
            image_.paste(image, mask=image.split()[3]) 
        else:
            image_ = image

        box["img"] = image_
        df.loc[box["id"]] = box


    df.set_index("id")
    return df

In [ ]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/reference_images_part1"
df = read_in_data(dir_str, ref_json)
df.head()